# Fine-tune MedGemma-4B with DPO

> 🗣️ [Large Language Model Course](https://github.com/mlabonne/llm-course)

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne).

You can run this notebook on Google Colab (A100).

In [ ]:
!pip install -qqq datasets trl peft bitsandbytes sentencepiece wandb --progress-bar off

In [1]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer, DPOConfig
import bitsandbytes as bnb
# from google.colab import userdata
import wandb

model_name = "google/medgemma-4b-it"
new_model = "medgemma-4b-it-medical-agent-dpo"

2025-07-19 17:51:43.708576: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752947503.731359     426 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752947503.738258     426 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import huggingface_hub as hf
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("hf_write_token")
WANDB_API_KEY = user_secrets.get_secret("WANDB_API_KEY")

hf.login(token=HF_TOKEN)
wandb.login(key=WANDB_API_KEY)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: wp29-04-2021-3months (wp29-04-2021-3months-home) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Format dataset

In [3]:
SYSTEM_PROMPT = """
You are a medical pre-diagnosis assistant. You will run in a loop of Thought, Action, PAUSE, Observation.

Thought: Analyze symptoms, context, and possible conditions.
Action: Choose one to run one of the actions available to you - then return PAUSE.
Observation: Results from your action (wait for this after each action).


Your available actions are:
- Search[symptoms/diseases]: Look up medical information by symptoms and/or relevant diseases (separate terms by comma).
- Ask[question]: Request symptom clarification from user.
- Finish[diagnosis]: Provide preliminary diagnosis with advice.

Important: Always use Search to verify medical information, even for familiar conditions, and all your actions input and output MUST be in Vietnamese.


Example session:

[User]
Question: Tôi bị đau đầu, sốt nhẹ và đau họng từ hôm qua. Tôi có thể đang bị bệnh gì?

[Your response]
Thought: Người dùng đang mô tả các triệu chứng đau đầu, sốt nhẹ và đau họng. Đây có thể là dấu hiệu của cảm cúm, viêm họng hoặc COVID-19. Tôi nên tìm kiếm thông tin về các bệnh này.
Action: Search: đau đầu, sốt nhẹ, đau họng, cảm cúm, viêm họng, COVID-19
PAUSE

[You will be called again with this]
Observation: Các triệu chứng đau đầu, sốt nhẹ và đau họng thường gặp trong nhiều bệnh lý như cảm cúm, viêm họng, COVID-19, và viêm xoang.

[Your response]
Thought: Tôi cần thêm thông tin về thời gian, mức độ triệu chứng và các triệu chứng đi kèm khác.
Action: Ask: Bạn có bị ho, chảy nước mũi hoặc khó thở không? Mức độ sốt của bạn là bao nhiêu?
PAUSE

[You will be called again with this]
Observation: Tôi có ho khan và chảy nước mũi nhẹ. Nhiệt độ khoảng 37.8°C.

[Your response]
Thought: Với các triệu chứng đau đầu, sốt nhẹ (37.8°C), đau họng, ho khan và chảy nước mũi nhẹ, đây rất có thể là triệu chứng của cảm cúm thông thường. Tôi nên tìm kiếm thông tin chi tiết về cảm cúm.
Action: Search: cảm cúm, ho khan, chảy nước mũi, đau đầu
PAUSE

[You will be called again with this]
Observation: Cảm cúm thông thường thường có các triệu chứng như ho khan, chảy nước mũi, đau đầu, đau họng và sốt nhẹ. Các triệu chứng thường kéo dài 7-10 ngày và có thể điều trị bằng nghỉ ngơi, uống nhiều nước và thuốc không kê đơn.

[Your response]
Thought: Các thông tin tìm kiếm xác nhận rằng các triệu chứng của người dùng phù hợp với cảm cúm thông thường.
Action: Finish: Dựa trên các triệu chứng bạn mô tả, bạn có thể đang bị cảm cúm thông thường. Khuyến nghị: nghỉ ngơi đầy đủ, uống nhiều nước, dùng thuốc hạ sốt như paracetamol nếu cần, và súc họng với nước muối ấm. Nếu triệu chứng kéo dài quá 5 ngày hoặc trở nên nghiêm trọng hơn (sốt cao trên 39°C, khó thở), hãy đi khám bác sĩ ngay.
"""

def add_system_prompt(example):
    example["prompt"].insert(0, {"role": "system", "content": SYSTEM_PROMPT}) 
    return example

dataset = load_dataset(
    "json", data_files="/kaggle/input/medical-agent-dpo-train-10-samples/dpo_train.json", split="train"
)
dataset = dataset.map(add_system_prompt)
dataset[0]

{'prompt': [{'content': '\nYou are a medical pre-diagnosis assistant. You will run in a loop of Thought, Action, PAUSE, Observation.\n\nThought: Analyze symptoms, context, and possible conditions.\nAction: Choose one to run one of the actions available to you - then return PAUSE.\nObservation: Results from your action (wait for this after each action).\n\n\nYour available actions are:\n- Search[symptoms/diseases]: Look up medical information by symptoms and/or relevant diseases (separate terms by comma).\n- Ask[question]: Request symptom clarification from user.\n- Finish[diagnosis]: Provide preliminary diagnosis with advice.\n\nImportant: Always use Search to verify medical information, even for familiar conditions, and all your actions input and output MUST be in Vietnamese.\n\n\nExample session:\n\n[User]\nQuestion: Tôi bị đau đầu, sốt nhẹ và đau họng từ hôm qua. Tôi có thể đang bị bệnh gì?\n\n[Your response]\nThought: Người dùng đang mô tả các triệu chứng đau đầu, sốt nhẹ và đau họ

## Train model with DPO

In [4]:
wandb.init(
    project="medgemma-4b-it-medical-agent-dpo", name="MedGemma 4B DPO on Medical ReAct Agent conversational data"
)

In [ ]:
# del dpo_trainer, model
gc.collect()
torch.cuda.empty_cache()

In [5]:
# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model.config.use_cache = False

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from trl import apply_chat_template

# def concat_prompt_full(example):
#     example["chosen"] = example["prompt"] + example["prompt"]
#     example["rejected"] = example["prompt"] + example["rejected"]
#     return example

def tokenize_function(example):
    prompt_ids = tokenizer.encode(example["prompt"])
    chosen_ids = prompt_ids + tokenizer.encode(example["chosen"], add_special_tokens=False)
    rejected_ids = prompt_ids + tokenizer.encode(example["rejected"], add_special_tokens=False)
    return {
        "prompt_len": len(prompt_ids),
        "output_len": max(len(chosen_ids), len(rejected_ids)),
    }

# extended = dataset.map(concat_prompt_full)
extended = dataset.map(apply_chat_template, fn_kwargs={"tokenizer": tokenizer})
token_len = extended.map(tokenize_function, remove_columns=extended.column_names)
# token_len
max(token_len["prompt_len"]), max(token_len["output_len"])

In [6]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    # target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
    target_modules=["q_proj", "v_proj"]
)

# Training arguments
training_args = DPOConfig(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    # max_steps=200,
    num_train_epochs=11,
    save_strategy="no",
    logging_steps=5,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="wandb",

    # DPO-specific
    beta=0.1,
    max_prompt_length=1024,
    max_length=1056,
)

model.gradient_checkpointing_enable()

# Create DPO trainer
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer,
    peft_config=peft_config,

)

# Fine-tune model with DPO
dpo_trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
5,0.703000
10,0.731500
15,0.703300
20,0.681200
25,0.653800
30,0.678900
35,0.648000
40,0.607200
45,0.497600
50,0.435000


TrainOutput(global_step=110, training_loss=0.3354566699666479, metrics={'train_runtime': 1211.3664, 'train_samples_per_second': 0.091, 'train_steps_per_second': 0.091, 'total_flos': 0.0, 'train_loss': 0.3354566699666479, 'epoch': 11.0})

In [7]:
wandb.finish()

train/epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇███
train/global_step,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇███
train/grad_norm,▆▅▆▅▅█▆▇▇▅▄▃▃▃▂▁▂▁▁▁▁▁
train/learning_rate,▁▁▂▂▃▃▃▄▄▄▅▅▅▆▆▇▇▇██▆▁
train/logits/chosen,▁▄▃▂▄▁▃▂▂▆▇▂▆▆▆█▆▇▅▇▇▄
train/logits/rejected,▇▁▄▅▇▂▅▄▅▆▆▇█▇██▇▇▅▆▃▅
train/logps/chosen,▄▃▂▄▆▁▃▄▄▄▇▂▇▄▅▇▅█▇▇█▆
train/logps/rejected,█▇█▇▇█▇█▇██▇▇▇▇▆▅▄▃▂▂▁
train/loss,████▇▇▇▇▆▅▅▃▃▂▁▂▁▁▁▁▁▁
train/rewards/accuracies,▁▁▄▄█▅████████████████
train/rewards/chosen,▁▁▁▁▁▁▁▂▂▂▂▄▄▅▆▅▆▇▇▇▆█


## Upload model

In [8]:
# Save artifacts
dpo_trainer.model.save_pretrained("final_checkpoint")
tokenizer.save_pretrained("final_checkpoint")

('final_checkpoint/tokenizer_config.json',
 'final_checkpoint/special_tokens_map.json',
 'final_checkpoint/chat_template.jinja',
 'final_checkpoint/tokenizer.model',
 'final_checkpoint/added_tokens.json',
 'final_checkpoint/tokenizer.json')

In [9]:
# Flush memory
del dpo_trainer, model
gc.collect()
torch.cuda.empty_cache()

# Reload model in BF16 (instead of NF4)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, "final_checkpoint")
model = model.merge_and_unload()

# Save model and tokenizer
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

# Push them to the HF Hub
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/nguyenit67/medgemma-4b-it-medical-agent-dpo/commit/9346b01cac721c2be7268a3dbc1f2f94902003aa', commit_message='Upload tokenizer', commit_description='', oid='9346b01cac721c2be7268a3dbc1f2f94902003aa', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nguyenit67/medgemma-4b-it-medical-agent-dpo', endpoint='https://huggingface.co', repo_type='model', repo_id='nguyenit67/medgemma-4b-it-medical-agent-dpo'), pr_revision=None, pr_num=None)

## Inference

In [ ]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
!nvidia-smi

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

new_model = "nguyenit67/medgemma-4b-it-medical-agent-dpo"
# Format prompt
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Hai ngày nay tôi bị sốt cao liên tục, ho không dứt, toàn thân đau mỏi và cảm thấy hoàn toàn không nếm được thức ăn."}
]
tokenizer = AutoTokenizer.from_pretrained(new_model)
prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

# Create pipeline
pipe = pipeline(
    "text-generation",
    model=new_model,
    tokenizer=tokenizer,
    device_map="auto"
)

In [ ]:
# Generate text
sequences = pipe(
    prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    max_length=100,
)
print(sequences[0]['generated_text'])